In [59]:
import math 
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
#url에 접속
html=urlopen('https://movie.naver.com/movie/running/current.nhn?order=reserve')
#html 분석기
bs=BeautifulSoup(html, 'html.parser') 
#태그 수집
items = bs.select('#content > div.article > div > div.lst_wrap > ul > li > dl > dt > a')
codes=[]
titles=[]
for item in items:
    # <a href=''>텍스트</a>  
    # href 속성에서 =을 기준으로 나눔
    code=(item['href'].split('=')[1])
    title=item.text 
    codes.append(code)
    titles.append(title)

In [60]:
import pandas as pd 
#데이터프레임으로 바꾸고
df=pd.DataFrame({'code':codes, 'title':titles})
df.head()

,code,title
0,163834,원더 우먼 1984
1,199393,조제
2,28876,화양연화
3,193194,도굴
4,195370,100% 울프: 푸들이 될 순 없어


In [61]:
#csv 파일로 저장
df.to_csv('c:/data/movie/movie_code.csv', encoding='ms949',index=False)

In [62]:
total_reviews=[]
total_points=[]
movie_codes=[]
movie_titles=[]
total_reviews2=[]
labels=[]
for j in range(len(codes[:10])):
    #수집할 url
    html=urlopen(f'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={codes[j]}&target=after&page=1')
    bs=BeautifulSoup(html, 'html.parser')
    # 태그 1개만 수집 select_one
    total=bs.select_one('#old_content > h5 > div > strong')
    #print(total)
    # 좌우 공백 제거, (), 제거
    #value=total.text.strip().replace('(','').replace(')','').replace(',','')
    pages=int(total.text)/10 
    pages=math.ceil(pages)  #올림
    # print('페이지수:',pages)

    #for i in range(1,pages+1):
    for i in range(1,20):
        # print(f'page {i}')
        url=f'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={codes[j]}&target=after&page={i}'
        webpage=urlopen(url)
        source=BeautifulSoup(webpage, 'html.parser')
        #영화리뷰텍스트
        reviews=source.select('tbody > tr > td.title')
        if i>pages: continue 
        for review in reviews:
            review = review.text.strip().split('\n\n')[2].replace('\t','').replace('\n','').replace('네이버','').replace('신고','')
            total_reviews.append(review)
            movie_codes.append(codes[j])
            movie_titles.append(titles[j])
        #영화평점
        points=source.select('td.title > div > em')
        for point in points:
            total_points.append(point.text) 
            # 8점 이상이면 1, 아니면 0으로 라벨링
            if int(point.text)>=8:
                labels.append(1)
            else:
                labels.append(0)

In [63]:
#영화코드,리뷰,평점,라벨 결합
items=list(zip(movie_codes, movie_titles, total_reviews, total_points, labels))
print(len(items))

1522


In [64]:
#데이터프레임으로 변환 및 저장
df=pd.DataFrame(items, columns=['code', 'title', 'review','point','label'])
df.to_csv('c:/data/movie/review.csv', encoding='utf-8', index=False)

In [65]:
df2=pd.read_csv('c:/data/movie/review.csv', encoding='utf-8')
df2

,code,title,review,point,label
0,163834,원더 우먼 1984,말해 뭐합니까! 겔가돗인데,10,1
1,163834,원더 우먼 1984,"눈호강,귀호강하고 왔습니다!",10,1
2,163834,원더 우먼 1984,페드로 파스칼 연기는 진짜 압권인데...아...개인적으론 처참 했다,1,0
3,163834,원더 우먼 1984,어렸을 때 봤던 초대 원더우먼 린다 카터가 쿠키에 나옵니다. 원더우먼 시리즈를 예전...,10,1
4,163834,원더 우먼 1984,너무 기대를 했는지 기대에는 못미쳤어요러닝타임이 쓸데없이 길고 편집이 좀 이상했어요,6,0
...,...,...,...,...,...
1517,19031,러브레터,이와이 월드의 시작~30주년 기념으로도 재개봉 해주면 좋겠네요,9,1
1518,19031,러브레터,"고3 겨울, 38 겨울을 잇는 영화...없던 감수성도 깊숙이 숨어버린 감수성도 솟게...",10,1
1519,19031,러브레터,"고3 겨울, 38 겨울을 잇는 영화...없던 감수성도 깊숙이 숨어버린 감수성도 솟게...",10,1
1520,19031,러브레터,킹시국이고 나발이고 방금 또극장가서 보고옴...다들 잘 살고 계시나요..?,10,1
